In [3]:
!pip install boto3
!pip install s3fs
from kafka import KafkaConsumer
from time import sleep
from json import dumps,loads
import json
from s3fs import S3FileSystem


  Using cached botocore-1.38.14-py3-none-any.whl.metadata (5.7 kB)
Using cached botocore-1.38.14-py3-none-any.whl (13.6 MB)
  Attempting uninstall: botocore
    Found existing installation: botocore 1.37.3
    Uninstalling botocore-1.37.3:
      Successfully uninstalled botocore-1.37.3


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.22.0 requires botocore<1.37.4,>=1.37.2, but you have botocore 1.38.14 which is incompatible.


  Using cached botocore-1.37.3-py3-none-any.whl.metadata (5.7 kB)
Using cached botocore-1.37.3-py3-none-any.whl (13.3 MB)
  Attempting uninstall: botocore
    Found existing installation: botocore 1.38.14
    Uninstalling botocore-1.38.14:
      Successfully uninstalled botocore-1.38.14


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
boto3 1.38.11 requires botocore<1.39.0,>=1.38.11, but you have botocore 1.37.3 which is incompatible.
s3transfer 0.12.0 requires botocore<2.0a.0,>=1.37.4, but you have botocore 1.37.3 which is incompatible.


In [9]:
consumer = KafkaConsumer(
    'demo_testing1',
    bootstrap_servers=['43.205.231.111:9092'],
    auto_offset_reset='earliest',
    enable_auto_commit=True,
    group_id='my-local-consumer',  # Use a unique group ID
    value_deserializer=lambda x: loads(x.decode('utf-8'))
)

In [ ]:
from kafka import KafkaConsumer
from json import loads

consumer = KafkaConsumer(
    'demo_testing1',
    bootstrap_servers=['43.205.231.111:9092'],
    auto_offset_reset='earliest',
    enable_auto_commit=True,
    group_id='my-local-consumer',
    value_deserializer=lambda x: x.decode('utf-8')  # defer JSON loading
)

for message in consumer:
    try:
        if message.value:
            data = loads(message.value)
            print(f"Received valid JSON: {data}")
        else:
            print("Received empty message.")
    except Exception as e:
        print(f"Invalid message received: {message.value}")
        print(f"Error: {e}")


Received valid JSON: {'message': 'Hello World'}
Received valid JSON: {'message': 'Hello World'}
Received valid JSON: {'message': 'Hello World'}
Received valid JSON: {'message': 'Hello World'}
Received valid JSON: {'message': 'Hello World'}


In [1]:
from kafka import KafkaConsumer
from json import loads
import json

# Create Kafka consumer
consumer = KafkaConsumer(
    'demo_testing1',
    bootstrap_servers=['13.201.71.173:9092'],
    auto_offset_reset='earliest',
    enable_auto_commit=True,
    group_id='my-local-consumer',
    value_deserializer=lambda x: x.decode('utf-8')  # Decode bytes to string
)

print("Listening for messages...")

for message in consumer:
    raw_value = message.value
    try:
        json_data = json.loads(raw_value)
        print(f"[JSON] Received: {json_data}")
    except json.JSONDecodeError:
        print(f"[RAW] Received non-JSON message: {raw_value}")


NoBrokersAvailable: NoBrokersAvailable

In [ ]:
#final code
from kafka import KafkaConsumer
from json import loads
import json

# Create Kafka consumer
consumer = KafkaConsumer(
    'demo_testing1',
    bootstrap_servers=['13.201.71.173:9092'],
    auto_offset_reset='earliest',
    enable_auto_commit=True,
    group_id='my-local-consumer',
    value_deserializer=lambda x: x.decode('utf-8')  # Decode bytes to string
)
s3=S3FileSystem()

In [ ]:
for count,i in enumerate(consumer):
    with s3.open('s3://kafka-stock-market-project-yeshwanth/stock_market_{}.json'.format(count),'w') as file:
        json.dump(i.value,file)

In [6]:
import boto3
from kafka import KafkaConsumer
import json

# Initialize the S3 client using boto3
s3_client = boto3.client('s3')
bucket_name = 'kafka-stock-market-analysis-yeshwanth'

# Create Kafka consumer
consumer = KafkaConsumer(
    'demo_testing1',
    bootstrap_servers=['13.201.71.173:9092'],
    auto_offset_reset='earliest',
    enable_auto_commit=True,
    group_id='my-local-consumer',
    value_deserializer=lambda x: x.decode('utf-8')  # Decode bytes to string
)

# Consumer loop to consume messages
for count, i in enumerate(consumer):
    print(f"[{count}] Raw Kafka message: {i.value!r}")  # Print the raw message for debugging

    try:
        # Parse the message to JSON
        parsed_json = json.loads(i.value)

        # Define the S3 object key (file name)
        object_key = f"stock_market_{count}.json"

        # Upload the parsed JSON to S3
        s3_client.put_object(
            Bucket=bucket_name,
            Key=object_key,
            Body=json.dumps(parsed_json),
            ContentType='application/json'
        )
        print(f"[{count}] Successfully uploaded to S3: {object_key}")

    except json.JSONDecodeError as e:
        print(f"[{count}] JSON decode error: {e}")
    except Exception as e:
        print(f"[{count}] Unexpected error: {e}")


[0] Raw Kafka message: 'This is Kafka Server !!!'
[0] JSON decode error: Expecting value: line 1 column 1 (char 0)
[1] Raw Kafka message: 'By Yeshwanth'
[1] JSON decode error: Expecting value: line 1 column 1 (char 0)
[2] Raw Kafka message: '{"message": "Hello World"}'
[2] Successfully uploaded to S3: stock_market_2.json
[3] Raw Kafka message: '{"message": "Hello World"}'
[3] Successfully uploaded to S3: stock_market_3.json
[4] Raw Kafka message: '{"message": "Hello World"}'
[4] Successfully uploaded to S3: stock_market_4.json
[5] Raw Kafka message: '{"message": "Hello World"}'
[5] Successfully uploaded to S3: stock_market_5.json
[6] Raw Kafka message: '{"message": "Hello World"}'
[6] Successfully uploaded to S3: stock_market_6.json
[7] Raw Kafka message: '{"Index": "HSI", "Date": "1990-12-31", "Open": 3053.0, "High": 3055.0, "Low": 3012.0, "Close": 3024.0, "Adj Close": 3024.0, "Volume": 0.0, "CloseUSD": 393.12}'
[7] Successfully uploaded to S3: stock_market_7.json
[8] Raw Kafka messag

KeyboardInterrupt: 

Deleted Successfully


In [ ]:
#final code
import boto3
from kafka import KafkaConsumer
import json

# Initialize the S3 client using boto3
s3_client = boto3.client('s3')
bucket_name = 'kafka-stock-market-analysis-yeshwanth'

# Create Kafka consumer
consumer = KafkaConsumer(
    'demo_testing1',
    bootstrap_servers=['13.233.92.65:9092'],
    auto_offset_reset='earliest',
    enable_auto_commit=True,
    group_id='my-local-consumer',
    value_deserializer=lambda x: x.decode('utf-8')  # Decode bytes to string
)

# Consumer loop to consume messages
for count, i in enumerate(consumer):
    print(f"[{count}] Consumed Kafka message: {i.value}")  # Print the consumed message
    
    try:
        # Check if the message is not empty
        if i.value:
            # Parse the message to JSON
            parsed_json = json.loads(i.value)

            # Check if parsed JSON is empty or malformed
            if not parsed_json:
                print(f"[{count}] Empty or malformed message: {i.value}")
                continue

            # Define the S3 object key (file name)
            object_key = f"stock_market_{count}.json"

            # Upload the parsed JSON to S3
            s3_client.put_object(
                Bucket=bucket_name,
                Key=object_key,
                Body=json.dumps(parsed_json),
                ContentType='application/json'
            )
            print(f"[{count}] Successfully uploaded to S3: {object_key}")
        else:
            print(f"[{count}] Empty message received from Kafka")

    except json.JSONDecodeError as e:
        print(f"[{count}] JSON decode error: {e} for message: {i.value}")
    except Exception as e:
        print(f"[{count}] Unexpected error: {e} for message: {i.value}")


[0] Consumed Kafka message: {"Index": "HSI", "Date": "31-12-1986", "Open": 2568.300048828125, "High": 2568.300048828125, "Low": 2568.300048828125, "Close": 2568.300048828125, "Adj Close": 2568.300048828125, "Volume": 0, "CloseUSD": 333.8790063476563}
[0] Successfully uploaded to S3: stock_market_0.json
[1] Consumed Kafka message: {"Index": "HSI", "Date": "02-01-1987", "Open": 2540.10009765625, "High": 2540.10009765625, "Low": 2540.10009765625, "Close": 2540.10009765625, "Adj Close": 2540.10009765625, "Volume": 0, "CloseUSD": 330.2130126953125}
[1] Successfully uploaded to S3: stock_market_1.json
[2] Consumed Kafka message: {"Index": "HSI", "Date": "31-12-1986", "Open": 2568.300048828125, "High": 2568.300048828125, "Low": 2568.300048828125, "Close": 2568.300048828125, "Adj Close": 2568.300048828125, "Volume": 0, "CloseUSD": 333.8790063476563}
[2] Successfully uploaded to S3: stock_market_2.json
[3] Consumed Kafka message: {"Index": "HSI", "Date": "02-01-1987", "Open": 2540.10009765625, 

In [ ]:
import boto3

# Initialize S3 client using boto3
s3_client = boto3.client('s3')

bucket_name = 'kafka-stock-market-analysis-yeshwanth'  # Replace with your bucket name

# Function to empty the bucket
def empty_s3_bucket(bucket_name):
    try:
        # List all objects in the bucket
        objects = s3_client.list_objects_v2(Bucket=bucket_name)

        # Check if there are objects in the bucket
        if 'Contents' in objects:
            print(f"Found {len(objects['Contents'])} objects in the bucket. Deleting them...")
            
            # Delete each object in the bucket
            for obj in objects['Contents']:
                object_key = obj['Key']
                print(f"Deleting object: {object_key}")
                s3_client.delete_object(Bucket=bucket_name, Key=object_key)
            
            print("All objects deleted successfully.")
        else:
            print("No objects found in the bucket.")

    except Exception as e:
        print(f"Error occurred: {e}")

# Call the function to empty the bucket
empty_s3_bucket(bucket_name)


In [ ]:
import boto3

# Initialize S3 client using boto3
s3_client = boto3.client('s3')

bucket_name = 'athena-query-results-yeshwanth-2025'  # Replace with your bucket name

# Function to empty the bucket
def empty_s3_bucket(bucket_name):
    try:
        # List all objects in the bucket
        objects = s3_client.list_objects_v2(Bucket=bucket_name)

        # Check if there are objects in the bucket
        if 'Contents' in objects:
            print(f"Found {len(objects['Contents'])} objects in the bucket. Deleting them...")
            
            # Delete each object in the bucket
            for obj in objects['Contents']:
                object_key = obj['Key']
                print(f"Deleting object: {object_key}")
                s3_client.delete_object(Bucket=bucket_name, Key=object_key)
            
            print("All objects deleted successfully.")
        else:
            print("No objects found in the bucket.")

    except Exception as e:
        print(f"Error occurred: {e}")

# Call the function to empty the bucket
empty_s3_bucket(bucket_name)
